<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/cdvae_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inverse Design of Next-Generation Superconductors Using Data-Driven Deep Generative Models

Reference: https://pubs.acs.org/doi/10.1021/acs.jpclett.3c01260

In [1]:
%%time
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q pytorch-lightning wandb torchmetrics==0.6.0 pymatgen==2022.4.26
!pip install -q hydra-core jarvis-tools python-dotenv p-tqdm accelerate

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 24.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# !pip install -q pytorch-lightning==1.3.6

In [3]:
import os
os.chdir('/content')
if not os.path.exists('cdvae'):
  !git clone https://github.com/JARVIS-Materials-Design/cdvae.git

os.chdir('cdvae')
!pip install -e .

Cloning into 'cdvae'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 227 (delta 77), reused 178 (delta 66), pack-reused 0
Receiving objects: 100% (227/227), 146.68 MiB | 13.75 MiB/s, done.
Resolving deltas: 100% (77/77), done.
Updating files: 100% (97/97), done.
Obtaining file:///content/cdvae
  Preparing metadata (setup.py) ... done
  Running setup.py develop for cdvae


In [4]:
import os
if not os.path.exists("/content/cdvae/WABDB"):
  os.makedirs("/content/cdvae/WABDB")
if not os.path.exists("/content/cdvae/HYDRA_JOBS"):
  os.makedirs("/content/cdvae/HYDRA_JOBS")
os.environ["PROJECT_ROOT"]="/content/cdvae"
os.environ["WABDB"]="/content/cdvae/WABDB"
os.environ["WABDB_DIR"]="/content/cdvae/WABDB"
os.environ["HYDRA_JOBS"]="/content/cdvae/HYDRA_JOBS"
%env HYDRA_FULL_ERROR=1


env: HYDRA_FULL_ERROR=1


In [5]:
!echo $HYDRA_FULL_ERROR

1


In [6]:
import yaml
import pprint
with open('/content/cdvae/conf/train/default.yaml','r') as f:
  yam = yaml.safe_load(f)
yam.pop('early_stopping')
#yam['pl_trainer']['fast_dev_run']=True
yam['pl_trainer']['gpus']=0

with open('/content/cdvae/conf/train/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

with open('/content/cdvae/conf/optim/default.yaml','r') as f:
  yam = yaml.safe_load(f)

yam['use_lr_scheduler']=False

with open('/content/cdvae/conf/optim/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

{'deterministic': False,
 'model_checkpoints': {'save_top_k': 1, 'verbose': False},
 'monitor_metric': 'val_loss',
 'monitor_metric_mode': 'min',
 'pl_trainer': {'accumulate_grad_batches': 1,
                'fast_dev_run': False,
                'gpus': 0,
                'gradient_clip_algorithm': 'value',
                'gradient_clip_val': 0.5,
                'max_epochs': '${data.train_max_epochs}',
                'num_sanity_val_steps': 2,
                'precision': 32,
                'profiler': 'simple'},
 'random_seed': 42}
{'lr_scheduler': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau',
                  'factor': 0.6,
                  'min_lr': '1e-4',
                  'patience': 30},
 'optimizer': {'_target_': 'torch.optim.Adam',
               'betas': [0.9, 0.999],
               'eps': '1e-08',
               'lr': 0.001,
               'weight_decay': 0},
 'use_lr_scheduler': False}


In [7]:
import os
os.environ["WANDB_ANONYMOUS"] = "must"
!python cdvae/run.py data=supercon expname=supercon_test02 model.predict_property=True

2023-07-28 00:58:54.118368: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-28 00:58:55.476354: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/run.py:164: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'default': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-p

In [8]:
!pip freeze

absl-py==1.4.0
accelerate==0.21.0
aiohttp==3.8.5
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
antlr4-python3-runtime==4.9.3
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
array-record==0.4.0
arviz==0.15.1
astropy==5.2.2
astunparse==1.6.3
async-timeout==4.0.2
attrs==23.1.0
audioread==3.0.0
autograd==1.6.2
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blinker==1.4
blis==0.7.10
blosc2==2.0.0
bokeh==3.1.1
branca==0.6.0
build==0.10.0
CacheControl==0.13.1
cachetools==5.3.1
catalogue==2.0.9
-e git+https://github.com/JARVIS-Materials-Design/cdvae.git@75f4637975920c977cec1ea5584f158a662f37e2#egg=cdvae
certifi==2023.7.22
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.6
click-plugins==1.1.1
cligj==0.7.2
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.1.0
cons==0.4.6
contextlib2==21.6.0
contourpy==1.1.0
convertdate==2.4.0
c